In [ ]:
!apt-get install swig cmake
!pip install setuptools==65.5.0

!pip install stable-baselines3[extra]

!pip install imageio
!pip install imageio_ffmpeg
!pip install pyglet==1.5.1
!apt-get install swig cmake
!pip install setuptools==65.5.0

!pip install stable-baselines3[extra]

!pip install imageio
!pip install imageio_ffmpeg
!pip install pyglet==1.5.1
!apt update
!apt install ffmpeg xvfb
!pip install xvfbwrapper
!pip install pyvirtualdisplay
!pip install gymnasium[box2d]


In [ ]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import A2C

from IPython import display
from IPython.display import HTML
import pygame
from base64 import b64encode
import matplotlib.pyplot as plt
import imageio
from time import sleep
import tqdm

import os

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

from stable_baselines3.common.callbacks import CheckpointCallback

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
env = gym.make("LunarLander-v2")

In [ ]:
#gradient_steps - How many gradient steps to do after each rollout (see train_freq) Set to -1 means to do as many gradient steps as steps done in the environment during the rollout.

model = A2C(
    'MlpPolicy',
    env,
    n_steps=1,
    gamma=0.85,
    learning_rate=0.001,
    verbose=1
    )

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
# Separate env for evaluation
eval_env = gym.make("LunarLander-v2")

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-133.97 +/- 67.23594916462083


In [ ]:

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path="./logs/",
  name_prefix="rl_model",
  save_replay_buffer=True,
  save_vecnormalize=True,
)

# Train the agent
model.learn(total_timesteps=5000, callback = checkpoint_callback)
# Save the agent
model.save("a2c_lunarlander")
#del model  # delete trained model to demonstrate loading

------------------------------------
| time/                 |          |
|    fps                | 184      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 100      |
| train/                |          |
|    entropy_loss       | -1.25    |
|    explained_variance | nan      |
|    learning_rate      | 0.001    |
|    n_updates          | 99       |
|    policy_loss        | 1.31     |
|    value_loss         | 2.54     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 152      |
|    ep_rew_mean        | -241     |
| time/                 |          |
|    fps                | 192      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 200      |
| train/                |          |
|    entropy_loss       | -1.27    |
|    explained_variance | nan      |
|    learning_rate      | 0.001    |
|

In [ ]:
model = A2C.load("a2c_lunarlander")


In [ ]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

In [ ]:
def convert_to_video(images, video_path):
  kargs = { 'macro_block_size': 1 }
  imageio.mimsave(video_path, [np.array(img) for i, img in enumerate(images)], fps=15, **kargs)

In [ ]:
env = gym.make('LunarLander-v2', render_mode = "rgb_array")
obs = env.reset()
images = []
obs = obs[0]
for i in range(2000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    img = env.render()
    images.append(img)
    # VecEnv resets automatically
    if done:
      obs = env.reset()
      obs = obs[0]

In [ ]:
video_path =  "replay.mp4"
convert_to_video(images, video_path)

In [ ]:
# Show video
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)